# Proyecto Final
## Behavioural Cloning

---

### Integrantes del Equipo 23:
* Carlos Pano Hernández - A01066264 [Campus Estado de México]
* Marie Kate Palau - A01705711 [Campus Monterrey]
* Edson Ulises Rodríguez Dávalos - A01796057 [Campus CdMx]
* Yohanna Ceballos Salomón - A01795115 [Campus Monterrey]

---

### Escuela de Ingeniería y Ciencias, Tecnológico de Monterrey
**Navegación autónoma (MR4010 - Gpo 10)**

---

#### Profesor Titular:
Dr. David Antonio Torres

#### Profesor Asistente:
Mtra. María Mylen Treviño Elizondo

---

**Martes 17 de junio del 2025**

# 1. Library Imports and Setup
This section contains all the necessary library imports for data manipulation, image processing, deep learning, and visualization that we'll use throughout this notebook.

In [35]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Lambda, Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import CosineSimilarity
import matplotlib.pyplot as plt

# Ruta a las imágenes
IMG_PATH = 'data/imagenes_capturadas'

# 1. Obtener dimensiones de la primera imagen
sample_image = Image.open(os.path.join(IMG_PATH, os.listdir(IMG_PATH)[0]))
IMG_WIDTH, IMG_HEIGHT = sample_image.size
NUM_CHANNELS = 3  # RGB

print(f"Image dimensions: {IMG_WIDTH}x{IMG_HEIGHT}")

# 2. Función para cargar, redimensionar, convertir a RGB y normalizar imágenes
def load_and_preprocess_image(file_path):
    img = Image.open(file_path).convert('RGB').resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = np.array(img) / 255.0
    return img_array

# 3. Cargar dataframe de entrenamiento
import pandas as pd
train_df = pd.read_csv('data/registro_conduccion.csv')  # Asegúrate que esta ruta sea correcta

# 4. Dividir en entrenamiento, validación y prueba
X = np.array([load_and_preprocess_image(os.path.join(IMG_PATH, fname)) for fname in train_df['nombre_imagen']])
y = train_df['angulo_direccion'].values

# División en entrenamiento, validación y prueba
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

# Para mantener proporciones en validación
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1765, random_state=42
)

print(f"Entrenamiento: {X_train.shape}, Validación: {X_val.shape}, Prueba: {X_test.shape}")

# 5. Crear objetos de data augmentation para entrenamiento y validación
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1
)

val_datagen = ImageDataGenerator()

# 6. Crear generadores
batch_size = 32
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

# 7. Crear la arquitectura del modelo inspirada en DAVE-2
model = Sequential()
model.add(Lambda(lambda x: x * 2. - 1., input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))  # rango (-1,1)
model.add(Conv2D(24, (5, 5), strides=(2, 2), activation='elu'))
model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='relu'))
model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='tanh'))  # Salida en rango [-1, 1]

# 8. Compilar el modelo con la métrica correcta
model.compile(
    loss='mse', 
    optimizer=Adam(learning_rate=1e-4), 
    metrics=[CosineSimilarity()]
)

# 9. Entrenar el modelo
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# 10. Evaluar en conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'\n✅ Test accuracy (cosine similarity): {test_acc:.4f}')
print(f'❌ Test loss (MSE): {test_loss:.4f}')

# 11. Plotear curvas de precisión y pérdida
plt.figure(figsize=(12, 5))

# Curva de precisión (cosine similarity) en entrenamiento y validación
plt.subplot(1, 2, 1)
plt.plot(history.history['cosine_similarity'], label='Train Cosine Similarity')
plt.plot(history.history['val_cosine_similarity'], label='Validation Cosine Similarity')
plt.title('Cosine Similarity over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Cosine Similarity')
plt.legend()

# Curva de pérdida en entrenamiento y validación
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# 1. Plot de predicciones vs valores reales
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.xlabel('Valor real')
plt.ylabel('Predicción')
plt.title('Predicciones vs Valores Reales')
plt.plot([-1,1], [-1,1], 'r--')  # línea de referencia
plt.grid(True)
plt.show()

# 2. Histograma del error (error = predicción - valor real)
errors = y_pred.flatten() - y_test
plt.figure(figsize=(8,4))
plt.hist(errors, bins=30, alpha=0.7)
plt.xlabel('Error (Predicción - Real)')
plt.ylabel('Frecuencia')
plt.title('Distribución del error')
plt.show()

# 3. Error absoluto medio por muestra (opcional: para análisis)
abs_errors = np.abs(errors)
plt.figure(figsize=(8,4))
plt.plot(abs_errors)
plt.xlabel('Índice de muestra')
plt.ylabel('Error absoluto')
plt.title('Error absoluto por muestra')
plt.show()

# 4. (Opcional) Crear una matriz tipo "errores discretizados"
# Por ejemplo, clasificar en 'error pequeño', 'error medio', 'error grande'
error_bins = [-np.inf, 0.1, 0.3, np.inf]
error_labels = ['pequeño', 'mediano', 'grande']
error_categories = np.digitize(abs_errors, bins=error_bins)
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(error_categories, error_categories, labels=[1, 2, 3])
labels = ['pequeño', 'mediano', 'grande']
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
plt.xlabel('Error Predicho')
plt.ylabel('Error Real')
plt.title('Matriz de confusión de errores discretizados')
plt.show()

# 12. Guardar el modelo entrenado
model.save('modelo_conduccion.h5')
print("Modelo guardado como 'modelo_conduccion.h5'")

# 13. Función para cargar y usar el modelo
def cargar_y_predecir(ruta_imagen):
    # Cargar el modelo con custom_objects para manejar la métrica MSE
    modelo_cargado = load_model('modelo_conduccion.h5', 
                              custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
    
    # Preprocesar la imagen
    img = load_and_preprocess_image(ruta_imagen)
    img = np.expand_dims(img, axis=0)  # Añadir dimensión de batch
    
    # Realizar predicción
    prediccion = modelo_cargado.predict(img)
    return prediccion[0][0]  # Retornar el ángulo predicho

# Obtener lista de imágenes en el directorio
imagenes = os.listdir(IMG_PATH)

# Predecir ángulo para las primeras 10 imágenes
for imagen in imagenes[:10]:  # Limitar a las primeras 10 imágenes
    if imagen.endswith(('.jpg', '.jpeg', '.png')):  # Solo procesar imágenes
        ruta_completa = os.path.join(IMG_PATH, imagen)
        angulo = cargar_y_predecir(ruta_completa)
        print(f"Imagen: {imagen}")
        print(f"Ángulo predicho: {angulo:.2f}°")
        print("-" * 50)


Image dimensions: 320x160
Entrenamiento: (306, 160, 320, 3), Validación: (66, 160, 320, 3), Prueba: (66, 160, 320, 3)
Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - cosine_similarity: -0.0741 - loss: 0.0063 - val_cosine_similarity: 0.1667 - val_loss: 0.0042
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - cosine_similarity: 0.2538 - loss: 0.0050 - val_cosine_similarity: 0.1667 - val_loss: 0.0043
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 177ms/step - cosine_similarity: 0.2504 - loss: 0.0057 - val_cosine_similarity: 0.1667 - val_loss: 0.0043
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 179ms/step - cosine_similarity: 0.1232 - loss: 0.0068 - val_cosine_similarity: 0.1667 - val_loss: 0.0042
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 180ms/step - cosine_similarity: 0.2149 - loss: 0.0046 - val_cosine_similarity: 0.1667 - val_loss: 0.0042
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 183ms/step - cosine_similarity: 0.2346 - loss: 0.0065 - val_cosine_similarity: 0.1667 - val_loss: 0.0042
Epoch

KeyboardInterrupt: 